# Insper - Supercomputação - Prova Intermediária

Questão sobre GPU / THRUST.


**Questão 4: Normalização de Vetor com Thrust**

Na análise de dados e no processamento de sinais, normalizar um vetor é uma operação comum que ajusta a escala dos elementos para que o vetor tenha uma norma unitária (ou seja, a soma dos quadrados dos elementos é igual a 1). Essa tarefa se torna computacionalmente intensiva para vetores de grande dimensão, fazendo com que a paralelização em GPU seja uma solução atrativa.

**Objetivo**:

Complemente o código abaixo usando a biblioteca Thrust para normalizar um vetor grande. O código inicial cria um vetor com valores aleatórios. Você deve implementar as etapas para:

1. calcular a norma L2 do vetor,
2. dividir cada elemento do vetor por essa norma e
3. imprimir o resultado final.

**Código Pré-Pronto**:

Complete o código com a sua solução.

In [8]:
%%writefile gpu.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/random.h>
#include <iostream>
#include <cmath>

// Função para gerar números aleatórios
struct prg
{
    float a, b;

    __host__ __device__
    prg(float _a=0.f, float _b=1.f) : a(_a), b(_b) {};

    __host__ __device__
    float operator()(const unsigned int n) const
    {
        thrust::default_random_engine rng;
        thrust::uniform_real_distribution<float> dist(a, b);
        rng.discard(n);

        return dist(rng);
    }
};

// Functor para calcular o quadrado de um número
struct square
{
    __host__ __device__
    float operator()(const float& x) const
    {
        return x * x;
    }
};

// Functor para dividir um número pela norma L2
struct divide_by_norm
{
    float norm;

    divide_by_norm(float _norm) : norm(_norm) {}

    __host__ __device__
    float operator()(const float& x) const
    {
        return x / norm;
    }
};

int main() {
    const int N = 1000000; // Tamanho do vetor
    thrust::counting_iterator<unsigned int> index_sequence_begin(0);
    thrust::host_vector<float> h_vec(N);

    // Preencher o vetor com números aleatórios entre 1.0 e 2.0
    thrust::transform(index_sequence_begin,
                      index_sequence_begin + N,
                      h_vec.begin(),
                      prg(1.f, 2.f));

    // Imprime alguns valores do vetor criado para verificação
    for(int i = 0; i < 20; i++) {
        std::cout << "Elemento original " << i << ": " << h_vec[i] << std::endl;
    }

    // Copia o vetor do host para o dispositivo
    thrust::device_vector<float> d_vec = h_vec;

    // Passo 1: Calcule o quadrado de cada elemento e armazene em um novo vetor
    thrust::device_vector<float> d_squared_vec(N);
    thrust::transform(d_vec.begin(), d_vec.end(), d_squared_vec.begin(), square());

    // Passo 2: Calcule a soma dos quadrados (usando reduce)
    float sum_of_squares = thrust::reduce(d_squared_vec.begin(), d_squared_vec.end(), 0.0f, thrust::plus<float>());

    // Passo 3: Tire a raiz quadrada da soma para obter a norma L2
    float norm = std::sqrt(sum_of_squares);

    // Imprime a norma calculada
    std::cout << "Norma calculada: " << norm << std::endl;

    // Normalize o vetor dividindo cada elemento pela norma L2
    thrust::transform(d_vec.begin(), d_vec.end(), d_vec.begin(), divide_by_norm(norm));

    // Imprime alguns valores do vetor normalizado para verificação
    for(int i = 0; i < 20; i++) {
        std::cout << "Elemento normalizado " << i << ": " << d_vec[i] << std::endl;
    }

    return 0;
}


Overwriting gpu.cu


Compilando o código

In [9]:
!nvcc -arch=sm_75 -std=c++14 gpu.cu -o gpu

Exemplo de execução:

In [10]:
%%time
!./gpu

Elemento original 0: 1.00002
Elemento original 1: 1.08503
Elemento original 2: 1.60135
Elemento original 3: 1.89161
Elemento original 4: 1.96796
Elemento original 5: 1.18969
Elemento original 6: 1.51498
Elemento original 7: 1.39801
Elemento original 8: 1.26291
Elemento original 9: 1.74351
Elemento original 10: 1.08955
Elemento original 11: 1.56039
Elemento original 12: 1.58223
Elemento original 13: 1.80957
Elemento original 14: 1.59192
Elemento original 15: 1.51171
Elemento original 16: 1.87663
Elemento original 17: 1.99508
Elemento original 18: 1.72621
Elemento original 19: 1.96661
Norma calculada: 1527.27
Elemento normalizado 0: 0.000654778
Elemento normalizado 1: 0.000710439
Elemento normalizado 2: 0.00104851
Elemento normalizado 3: 0.00123856
Elemento normalizado 4: 0.00128854
Elemento normalizado 5: 0.000778965
Elemento normalizado 6: 0.00099195
Elemento normalizado 7: 0.000915364
Elemento normalizado 8: 0.000826904
Elemento normalizado 9: 0.00114159
Elemento normalizado 10: 0.000